
# Fine-tuning

This tutorial explains how to fine-tune the `GR00T-N1` pretrained checkpoint on a post-training dataset of the same embodiment. This demonstrates the benefits of post-training, transforming a general model into a specialist model and showing performance improvements.

In this tutorial, we will use the demonstration dataset `robot_sim.PickNPlace` from the [demo_data](./demo_data) folder.

We will first load the pretrained model and evaluate it on the dataset. Then we will fine-tune the model on the dataset and evaluate its performance.

## Pretrained Model

In [ ]:
from gr00t.utils.eval import calc_mse_for_single_trajectory
import warnings
from gr00t.experiment.data_config import DATA_CONFIG_MAP
from gr00t.model.policy import Gr00tPolicy
from gr00t.data.schema import EmbodimentTag
from gr00t.data.dataset import LeRobotSingleDataset
import numpy as np
import torch

device = "cuda" if torch.cuda.is_available() else "cpu"

warnings.simplefilter("ignore", category=FutureWarning)

In [ ]:
PRE_TRAINED_MODEL_PATH = "nvidia/GR00T-N1-2B"
EMBODIMENT_TAG = EmbodimentTag.GR1
DATASET_PATH = "../demo_data/robot_sim.PickNPlace"


data_config = DATA_CONFIG_MAP["gr1_arms_only"]
modality_config = data_config.modality_config()
modality_transform = data_config.transform()


pre_trained_policy = Gr00tPolicy(
    model_path=PRE_TRAINED_MODEL_PATH,
    embodiment_tag=EMBODIMENT_TAG,
    modality_config=modality_config,
    modality_transform=modality_transform,
    device=device,
)

dataset = LeRobotSingleDataset(
    dataset_path=DATASET_PATH,
    modality_configs=modality_config,
    video_backend="decord",
    video_backend_kwargs=None,
    transforms=None,  # We'll handle transforms separately through the policy
    embodiment_tag=EMBODIMENT_TAG,
)


mse = calc_mse_for_single_trajectory(
    pre_trained_policy,
    dataset,
    traj_id=0,
    modality_keys=["right_arm", "right_hand"],   # we will only evaluate the right arm and right hand
    steps=150,
    action_horizon=16,
    plot=True
)

print("MSE loss for trajectory 0:", mse)

Great! We can see the predicted actions and the ground truth actions. The predicted actions are not perfect, but they are close to the ground truth actions. This indicates that the pretrained checkpoint works well.

Now let's randomly sample 10 trajectories and calculate the average MSE to get more detailed results.

In [ ]:
total_trajectories = len(dataset.trajectory_lengths)

print("Total trajectories:", total_trajectories)

sampled_trajectories = np.random.choice(total_trajectories, 10)
print("Sampled trajectories:", sampled_trajectories)

all_mses = []

for traj_id in sampled_trajectories:
    mse = calc_mse_for_single_trajectory(
        pre_trained_policy,
        dataset,
        traj_id=traj_id,
        modality_keys=["right_arm", "right_hand"],   # we will only evaluate the right arm and right hand
        steps=150,
        action_horizon=16,
        plot=False
    )
    print(f"Trajectory {traj_id} MSE: {mse:.4f}")
    
    all_mses.append(mse)

print("====================================")
print("Mean MSE:", np.mean(all_mses))
print("Std MSE:", np.std(all_mses))


## Fine-Tuning the Model

We’re now ready to fine-tune the model on the dataset. Without diving into the nitty-gritty of the fine-tuning process, we’ll simply use the `gr00t_finetune.py` script. Run the command below to get started:

```bash
python scripts/gr00t_finetune.py \
  --dataset-path ./demo_data/robot_sim.PickNPlace \
  --num-gpus 1 \
  --max-steps 500 \
  --output-dir /tmp/gr00t-1/finetuned-model \
  --data-config gr1_arms_only
```

_To see all available options, run `python scripts/gr00t_finetune.py --help`._

The script will save the fine-tuned model in `/tmp/gr00t-1/finetuned-model`. We’ll later load the checkpoint saved at **step 500**.

### Fine-Tuning Evaluation

Now we can evaluate the fine-tuned model by running the policy on the dataset to see how well it performs. We’ll use a utility function to assess the policy on the dataset—similar to the earlier tutorial [1_pretrained_model.ipynb](1_pretrained_model.ipynb).

In [ ]:
finetuned_model_path = "/tmp/gr00t-1/finetuned-model/checkpoint-500"

from gr00t.utils.eval import calc_mse_for_single_trajectory
import warnings

finetuned_policy = Gr00tPolicy(
    model_path=finetuned_model_path,
    embodiment_tag="new_embodiment",
    modality_config=modality_config,
    modality_transform=modality_transform,
    device=device,
)

warnings.simplefilter("ignore", category=FutureWarning)

mse = calc_mse_for_single_trajectory(
    finetuned_policy,
    dataset,
    traj_id=0,
    modality_keys=["right_arm", "right_hand"],   # we will only evaluate the right arm and right hand
    steps=150,
    action_horizon=16,
    plot=True
)

print("MSE loss for trajectory 0:", mse)

Awesome! We’ve fine-tuned the model and evaluated it on the dataset. The results show it has mastered the task and now outperforms the pre-trained baseline.